In [8]:
from alice import Alice
from bob import Bob

def execute_qkd(pairs, DEBUG = True, depolarize_probability = 0):
    # Initialization
    alice = Alice(pairs)
    bob = Bob(depolarize_probability)
    
    # Send pairs
    pairs = alice.prepare()

    # Measure each pair and retrieve double matchings
    double_matchings = bob.measure(pairs)
    
    # Verify that there's double matchings
    if not len(double_matchings):
        print("Key exchange failed, there's no double matchings. Please, try again.")
        return (), ()
    
    # Compute usable frames for Bob
    usable_frames, usable_frames_types = alice.compute_usable_frames(double_matchings)
    
    # Veirfy that there's usable frames
    if not len(usable_frames):
        print("Key exchange failed, there's no usable frames. Please, try again.")
        return (), ()

    # Compute sifting string
    sifting_string = bob.compute_sifting_string(usable_frames)

    # Compute measured string
    measured_string = bob.compute_measured_string(usable_frames)

    # Bob computes secret key
    bob_key = bob.generate_shared_key(usable_frames)

    # Alice computes secret key
    alice_key = alice.generate_shared_key(usable_frames_types, sifting_string)
    
    if alice_key == bob_key:
        if DEBUG:
            print("Key exchange completed, here's the data:")
            print(f"Alice send pairs: {alice.pairs_data}")
            print(f"Bob send double matchings: {double_matchings}")
            print(f"Alice send usable frames: {usable_frames}")
            print(f"Bob send sifting string: {sifting_string}")
            print(f"Bob shared key: {bob_key}")
            print(f"Alice shared key: {alice_key}")
        return (usable_frames, sifting_string, measured_string), (alice_key, bob_key)
    else:
        print("Key exchange failed, both keys are not equal. Please, try again.")
        return (usable_frames, sifting_string, measured_string), (alice_key, bob_key)

In [9]:
public, private = execute_qkd(16)

Key exchange completed, here's the data:
Alice send pairs: {0: '0x,0z', 1: '1x,0z', 2: '1x,0z', 3: '0x,1z', 4: '0x,1z', 5: '0x,1z', 6: '1x,1z', 7: '0x,0z', 8: '1x,0z', 9: '1x,0z', 10: '1x,0z', 11: '0x,0z', 12: '1x,0z', 13: '0x,1z', 14: '1x,1z', 15: '0x,1z'}
Bob send double matchings: [1, 4, 8, 11, 12, 14]
Alice send usable frames: [(1, 4), (1, 14), (4, 8), (4, 12), (4, 14), (8, 14), (12, 14)]
Bob send sifting string: ['11', '11', '11', '01', '00', '11', '01']
Bob shared key: 10101101011001
Alice shared key: 10101101011001


In [3]:
from partial_key_recovery import attack

public, private = execute_qkd(256, DEBUG = False)
usable_frames, sifting_string, conjugate_sifting_string = public
alice_key, bob_key = private
assert alice_key == bob_key

key_recovered = attack(usable_frames, sifting_string, conjugate_sifting_string, DEBUG = False)

private_key_blocks = [ alice_key[i : i + 2] for i in range(0, len(private[0]), 2)  ]
key_recovered_blocks = [ key_recovered[i : i + 2] for i in range(0, len(key_recovered), 2)  ]

'''
for x, y in zip(private_key_blocks, key_recovered_blocks):
    if x != y:
        print(x, y)
'''

for i in range(len(private_key_blocks)):
    if private_key_blocks[i] != key_recovered_blocks[i]:
        print(usable_frames[i], key_recovered_blocks[i], private_key_blocks[i])

In [6]:
def key_recovery_metrics(bits):
    public, private = execute_qkd(bits, DEBUG = False)
    usable_frames, sifting_string, conjugate_sifting_string = public
    alice_key, bob_key = private
    assert alice_key == bob_key

    key_recovered = attack(usable_frames, sifting_string, conjugate_sifting_string, DEBUG = False)

    private_key_blocks = [ alice_key[i : i + 2] for i in range(0, len(private[0]), 2)  ]
    key_recovered_blocks = [ key_recovered[i : i + 2] for i in range(0, len(key_recovered), 2)  ]

    bits_recovered = 0
    for i in range(len(private_key_blocks)):
        if private_key_blocks[i] == key_recovered_blocks[i]:
            bits_recovered += 2
    
    return bits_recovered / len(alice_key)

In [44]:
from numpy import mean

mean_recovered_per = []
bits = 32

for _ in range(1000):
    recovered_per = key_recovery_metrics(bits)
    mean_recovered_per.append(recovered_per)
    
mean(mean_recovered_per)

0.984

In [12]:
from numpy import mean

# mean_recovered_per = []
bits = 256

for _ in range(1000):
    recovered_per = key_recovery_metrics(bits)
    mean_recovered_per.append(recovered_per)
    
mean(mean_recovered_per)

0.7510360863453527